<a href="https://colab.research.google.com/github/b4lbs/alura_ML_validacao_de_modelos/blob/main/ML_valida%C3%A7%C3%A3o_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
X = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
SEED = 42
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify= y)
print(f'Treinaremos com {len(X_train)} elementos e testaremos com {len(X_test)} elementos.')

Treinaremos com 7500 elementos e testaremos com 2500 elementos.


In [ ]:
from sklearn.dummy import DummyClassifier
dummy_stratified = DummyClassifier()
dummy_stratified.fit(X_train, y_train)
acuracia = dummy_stratified.score(X_test, y_test) * 100
print(f'A acurácia do dummy stratified foi {acuracia:.2f}')

A acurácia do dummy stratified foi 58.00


In [ ]:
from sklearn.tree import DecisionTreeClassifier
SEED = 42
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)
acuracia = accuracy_score(y_test, previsoes) * 100
print(f'A acurácia da Árvore de Decisão {acuracia:.2f}')

A acurácia da Árvore de Decisão 75.12


OBSERVANDO A INFLUÊNCIA DA ALEATORIEDADE NA VALIDAÇÃO DO MODELO

In [ ]:
X = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

SEED = 2001
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify= y)
print(f'Treinaremos com {len(X_train)} elementos e testaremos com {len(X_test)} elementos.')

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acuracia = accuracy_score(y_test, previsoes) * 100
print(f'A acurácia da Árvore de Decisão {acuracia:.2f}')

Treinaremos com 7500 elementos e testaremos com 2500 elementos.
A acurácia da Árvore de Decisão 74.88


ACURÁCIAS COM DIFERENTES SEEDS:


42 - 75.12%


5 - 76.84%

2001 - 74.88%


Com as porcentagens acima podemos concluir que uma decisão de validação não deve ser tomada com base na aleatoriedade do processo, dando gancho à validação cruzada.

In [ ]:
from sklearn.model_selection import cross_validate

#A definição da seed será usada apenas pelo DecisionTreeClassifier
#pois o cv do cross_validate utiliza o KFold com o parâmetro shuffle como "False"
SEED = 42
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=3)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print(f'Accuracy com cross validation 3: {(media - 2 * desvio_padrao) * 100:.2f}% - {(media + 2 * desvio_padrao) * 100 :.2f}%')

Accuracy com cross validation 3: 74.99% - 76.57%


In [ ]:
#CV=5 é o valor default do cross_validate
#INTERESSANTE USAR CV IGUAL DE 5 A 10 OU DE 7+-2.
SEED = 42
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=5)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print(f'Accuracy com cross validation 5: {(media - 2 * desvio_padrao) * 100:.2f}% - {(media + 2 * desvio_padrao) * 100 :.2f}%')

Accuracy com cross validation 5: 75.21% - 76.35%


In [ ]:
SEED = 42
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=5)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print(f'Accuracy com cross validation 5: {(media - 2 * desvio_padrao) * 100:.2f}% - {(media + 2 * desvio_padrao) * 100 :.2f}%')

Accuracy com cross validation 5: 75.21% - 76.35%


In [ ]:
def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print(f'Accuracy médio: {media * 100:.2f}%')
  print(f'Accuracy intervalo: {(media - 2 * desvio_padrao) * 100:.2f}% - {(media + 2 * desvio_padrao) * 100 :.2f}%')

In [ ]:
#"Por trás dos panos do cross_validate"
from sklearn.model_selection import KFold

SEED = 42
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv = cv)
imprime_resultados(results)

Accuracy médio: 75.78%
Accuracy intervalo: 74.37% - 77.19%


In [ ]:
#Trazendo o aspecto de aleatoriedade para a validação cruzada

SEED = 42
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv = cv)
imprime_resultados(results)

Accuracy médio: 75.76%
Accuracy intervalo: 72.76% - 78.76%


O cross_validate aceita CV com diversos outros Splitter Classes

Documentação: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

Splitter Classes: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection


In [ ]:
#Usando o StratifiedKfold para trabalhar em casos onde há o desbalanceamento das classes
from sklearn.model_selection import StratifiedKFold

SEED = 42
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv = cv)
imprime_resultados(results)

Accuracy médio: 75.78%
Accuracy intervalo: 72.44% - 79.12%


In [ ]:
#Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar o estimador
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size = 10000) #Criando a coluna com base na idade do modelo + número aleatório entre -2 e 2
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1 #Corrigindo a coluna para evitar modelo negativo ou 0
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,21
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,7
4,117384.68,1,4,12770.11290,8


In [ ]:
dados.modelo.unique()

array([21, 24, 14,  7,  8, 12, 18, 19,  5, 13, 16, 17, 15, 11,  3, 10, 22,
       20, 23,  9,  4,  2,  6,  1])

In [ ]:
dados.modelo.value_counts()

20    840
19    826
18    801
21    749
16    672
17    660
14    626
15    600
22    597
13    541
12    470
11    449
10    387
23    371
9     296
8     257
7     206
24    188
6     175
5     154
4      66
3      45
2      18
1       6
Name: modelo, dtype: int64

In [ ]:
#Validação cruzada com grupos
from sklearn.model_selection import GroupKFold

SEED = 42
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy médio: 75.77%
Accuracy intervalo: 72.29% - 79.26%


CROSS VALIDATION COM STANDARDSCALER

In [ ]:
#MODELO COM TRAIN E TEST, NÃO POSSUI CROSS VALIDATION
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 42
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo = SVC()
modelo.fit(X_train_scaled, y_train)
previsoes = modelo.predict(X_test_scaled)
acuracia = accuracy_score(y_test, previsoes) * 100
print(f'A acurácia da SVC foi {acuracia:.2f}')

A acurácia da SVC foi 76.28


In [ ]:
#Utilizando PIPELINE para realizar o processo de transformação e previsão para cada validação
from sklearn.pipeline import Pipeline

SEED = 42
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, X, y, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy médio: 76.68%
Accuracy intervalo: 74.25% - 79.11%
